In [130]:
import cv2 as cv
import numpy as np
import random
import matplotlib.pyplot as plt
from osgeo import gdal,ogr,osr
from tqdm import tqdm

In [127]:
id_mosaico = '179323' 
file_shp_1 = '../../daninhas/ortomosaicos/pre-emergente/' + id_mosaico + '/daninha_1.shp'
file_shp_2 = '../../daninhas/ortomosaicos/pre-emergente/' + id_mosaico + '/daninha_2.shp'
name_mosaico = '../../daninhas/ortomosaicos/pre-emergente/'+ id_mosaico + '/ortomosaico.tif'
#output_dataset = '../../daninhas/ortomosaicos/pre-emergente/' + id_mosaico #Pasta de saida
output_dataset = '../../daninhas/dataset'
tif_generated_1 = '../../daninhas/ortomosaicos/pre-emergente/' + id_mosaico + '/daninha_1.tif' #Tip de saida
tif_generated_2 = '../../daninhas/ortomosaicos/pre-emergente/' + id_mosaico + '/daninha_2.tif'

MAX_PIXELS_DANINHA = 5000

In [3]:
def GetExtent(gt,cols,rows):
    ''' Return list of corner coordinates from a geotransform

        @type gt:   C{tuple/list}
        @param gt: geotransform
        @type cols:   C{int}
        @param cols: number of columns in the dataset
        @type rows:   C{int}
        @param rows: number of rows in the dataset
        @rtype:    C{[float,...,float]}
        @return:   coordinates of each corner
    '''
    ext=[]
    xarr=[0,cols]
    yarr=[0,rows]

    for px in xarr:
        for py in yarr:
            x=gt[0]+(px*gt[1])+(py*gt[2])
            y=gt[3]+(px*gt[4])+(py*gt[5])
            ext.append([x,y])
            print(x,y)
        yarr.reverse()
    return ext

In [4]:
def ReprojectCoords(coords,src_srs,tgt_srs):
    ''' Reproject a list of x,y coordinates.

        @type geom:     C{tuple/list}
        @param geom:    List of [[x,y],...[x,y]] coordinates
        @type src_srs:  C{osr.SpatialReference}
        @param src_srs: OSR SpatialReference object
        @type tgt_srs:  C{osr.SpatialReference}
        @param tgt_srs: OSR SpatialReference object
        @rtype:         C{tuple/list}
        @return:        List of transformed [[x,y],...[x,y]] coordinates
    '''
    trans_coords=[]
    transform = osr.CoordinateTransformation( src_srs, tgt_srs)
    for x,y in coords:
        x,y,z = transform.TransformPoint(x,y)
        trans_coords.append([x,y])
    return trans_coords

In [5]:
ds=gdal.Open(name_mosaico)
gt=ds.GetGeoTransform()
cols = ds.RasterXSize
rows = ds.RasterYSize
ext=GetExtent(gt,cols,rows)

src_srs=osr.SpatialReference()
src_srs.ImportFromWkt(ds.GetProjection())
tgt_srs = src_srs.CloneGeogCS()

geo_ext= ReprojectCoords(ext,src_srs,tgt_srs)

xmin = geo_ext[0][0]
ymax = geo_ext[0][1]
xmax = geo_ext[2][0]
ymin = geo_ext[1][1]

pixel_size_x = gt[1]
pixel_size_y = gt[5]

-45.4861271872156 -12.89707810163661
-45.4861271872156 -12.907532193067741
-45.475512627790565 -12.907532193067741
-45.475512627790565 -12.89707810163661


In [11]:
!gdal_rasterize -burn 255 -ot Byte -tr $pixel_size_x $pixel_size_y -te $xmin $ymin $xmax $ymax $file_shp_1 $tif_generated_1

#!gdal_rasterize -burn 255 -ot Byte -tr $pixel_size_x $pixel_size_y -te $xmin $ymin $xmax $ymax $file_shp_2 $tif_generated_2

0...10...20...30...40...50...60...70...80...90...100 - done.


In [6]:
daninha_1 = gdal.Open(tif_generated_1)
daninha_band_1 = daninha_1.GetRasterBand(1)

#daninha_2 = gdal.Open(tif_generated_2)
#daninha_band_2 = daninha_2.GetRasterBand(1)

mosaico = gdal.Open(name_mosaico)

red = mosaico.GetRasterBand(1)
green = mosaico.GetRasterBand(2)
blue = mosaico.GetRasterBand(3)
alpha = mosaico.GetRasterBand(4)


In [100]:
def random_func(*, percentage: int) -> int:
    '''
    percentage: Valores entre 0 e 100. Caso value > percentag é retornado true, caso não
    é retornado false
    
    description: Utilizada para definir se a função de pré processamento será utilizada ou
    não. 
    
    return:
    bool
    '''
    _value = random.randint(0, 100)
    if _value < percentage:
        #print(_value)
        return False
    else:
        #print(_value)
        return True

In [126]:
z = random_func(percentage = 10)
print(z)

True


In [132]:
total_images = 10000 # Total de imagens desejadas (treino e validacao)
count_yes = 0 # contador para as imagens com sim
count_no = 0 # contador para as imagens com sim
count_validation = 0
iterator_x = 256 # incremento no eixo x
iterator_y = 256 # incremento no eixo y
expected_yes_perc = 1 # porcentagem entre a quantidade de imagens sim e nao desejadas
expected_validation_perc = 0.25 # porcentagem entre a quantidade de imagens de validacao
black_background = 0.5 # porcentagem de fundo preto aceitavel na imagem


for x in range(0,mosaico.RasterXSize,iterator_x):
    for y in range(0,mosaico.RasterYSize,iterator_y):
        if count_yes + count_no >= total_images:
            break
            
        if ((x+iterator_x)>mosaico.RasterXSize) or ((y+iterator_y)>mosaico.RasterYSize):
            continue
            
        daninha_parcela = daninha_band_1.ReadAsArray(x,y,iterator_x,iterator_y) #+ daninha_band_2.ReadAsArray(x,y,100,100)
        blueparcela = blue.ReadAsArray(x,y,iterator_x,iterator_y)
        greenparcela = green.ReadAsArray(x,y,iterator_x,iterator_y)
        redparcela = red.ReadAsArray(x,y,iterator_x,iterator_y)
        imgparcela = cv.merge((blueparcela,greenparcela,redparcela))
        
        hsv = cv.cvtColor(imgparcela, cv.COLOR_BGR2HSV)
        mask = cv.inRange(hsv, (25, 0, 0), (50, 255,255))
        imask = mask>0
        green_filter = np.zeros_like(imgparcela, np.uint8)
        green_filter[imask] = imgparcela[imask]
        (channel_h, channel_s, channel_v) = cv.split(green_filter)
        retval, threshold = cv.threshold(channel_v, 130, 255, cv.THRESH_BINARY+cv.THRESH_OTSU)
        
        if (cv.countNonZero(blueparcela) + cv.countNonZero(greenparcela) + cv.countNonZero(redparcela)) < int(iterator_x * iterator_y * black_background * 3):
            continue
            
        if cv.countNonZero(daninha_parcela) > 0 and count_yes < total_images * expected_yes_perc: and cv.countNonZero(channel_v) > 100:
            if count_validation < total_images * expected_validation_perc and not random_func(percentage = 10):
                cv.imwrite(output_dataset + '/validation/sim/daninha_{x}_{y}.png'.format(x=x,y=y), imgparcela)
                cv.imwrite(output_dataset + '/mask_validation/sim/daninha_{x}_{y}.png'.format(x=x,y=y), threshold)
                count_validation += 1
                
            else:
                cv.imwrite(output_dataset + '/train/sim/daninha_{x}_{y}.png'.format(x=x,y=y), imgparcela)
                cv.imwrite(output_dataset + '/mask_train/sim/aninha_{x}_{y}.png'.format(x=x,y=y), threshold)
            
            count_yes += 1
        
        #elif cv.countNonZero(daninha_parcela) == 0 and count_no < total_images * (1 - expected_yes_perc) and cv.countNonZero(channel_v) == 0:
        #    cv.imwrite(output_dataset + '/nao/daninha_{x}_{y}.png'.format(x=x,y=y), imgparcela)
        #    cv.imwrite(output_dataset + '/mask_nao/daninha_{x}_{y}.png'.format(x=x,y=y), threshold)
        #    count_no += 1

In [142]:
print(y*x / (256*256))

8010.0
